In [5]:
import pandas as pd

In [8]:
df_estados = pd.read_csv('work/raw_data/estado_regiao.csv', encoding='iso-8859-1', delimiter=';')

In [9]:
df_estados.head()

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil


In [12]:
df_respondentes = pd.read_csv('work/raw_data/respondentes_pesquisa.csv', encoding='iso-8859-1', delimiter='|', decimal='.')

In [13]:
df_respondentes.head()

,cod_pessoa,genero,data_nascimento,educacao,renda_mensal,peso,colesterol,estado_civil,id_estado
0,10141743,Masculino,1981-04-25,Pós-Graduação,1000-3000,61.018446,96.777322,Viúvo,19
1,10141860,Feminino,1966-01-19,Ensino Médio,> 5000,108.557401,176.108130,Viúvo,1
2,10142548,Masculino,1973-08-02,Pós-Graduação,> 5000,112.349921,144.110224,Casado,19
3,10142591,Masculino,1997-03-05,Ensino Fundamental,< 1000,116.238829,235.386072,Divorciado,17
4,10142654,Feminino,1969-08-05,Graduação,3000-5000,88.609912,38.102665,Casado,15


In [17]:
import os

PATH = 'work/raw_data/dados_pesquisa_preferencias/'
lista_df = [] # cria uma lista vazia
for arquivo in os.listdir(PATH):
    local_arquivo = os.path.join(PATH, arquivo)
    if arquivo.endswith('.csv'):
        df = pd.read_csv(local_arquivo, sep = ';', encoding='latin-1')       
        lista_df.append(df)

df_preferencias = pd.concat(lista_df)

In [18]:
df_preferencias.head()

,cod_pessoa,data_coleta,animal_estimacao,clima,bebida_favorita,hobby,atividade_fisica,preferencia_musical
0,10142193,2024-08-02,Pássaro,moderado,refrigerante,Escutar música,Nunca,Rock
1,10142717,2024-08-02,Peixe,quente,Cerveja,Praticar esporte,Mensal,Pop
2,10143283,2024-08-02,Pássaro,frio,Chá,Assistir TV,Diária,Sertanejo
3,10143497,2024-08-02,Cachorro,quente,Café,Dormir,Semanal,Clássica
4,10143612,2024-08-02,Peixe,frio,Água,Praticar esporte,Semanal,Rock


In [30]:
df_unido = pd.merge(df_respondentes, df_estados, left_on='id_estado', right_on='id_estado', how='left').drop('id_estado', axis=1)
df_unido = pd.merge(df_respondentes, df_preferencias, left_on='cod_pessoa', right_on='cod_pessoa', how='left')

In [31]:
df_unido.head()

,cod_pessoa,genero,data_nascimento,educacao,renda_mensal,peso,colesterol,estado_civil,id_estado,data_coleta,animal_estimacao,clima,bebida_favorita,hobby,atividade_fisica,preferencia_musical
0,10141743,Masculino,1981-04-25,Pós-Graduação,1000-3000,61.018446,96.777322,Viúvo,19,2024-01-05,Lagarto,frio,Cerveja,Praticar esporte,Diária,Pop
1,10141860,Feminino,1966-01-19,Ensino Médio,> 5000,108.557401,176.108130,Viúvo,1,2024-01-05,Coelho,quente,Café,Assistir TV,Semanal,Pop
2,10142548,Masculino,1973-08-02,Pós-Graduação,> 5000,112.349921,144.110224,Casado,19,2024-01-05,Lagarto,frio,Café,Ler livros,Nunca,Rock
3,10142591,Masculino,1997-03-05,Ensino Fundamental,< 1000,116.238829,235.386072,Divorciado,17,2024-01-05,Pássaro,moderado,Cerveja,Aprender algo novo,Mensal,Rock
4,10142654,Feminino,1969-08-05,Graduação,3000-5000,88.609912,38.102665,Casado,15,2024-01-05,Lagarto,moderado,Café,Assistir TV,Nunca,Pop


In [33]:
df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2195 entries, 0 to 2194
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cod_pessoa           2195 non-null   int64  
 1   genero               2195 non-null   object 
 2   data_nascimento      2195 non-null   object 
 3   educacao             2195 non-null   object 
 4   renda_mensal         2195 non-null   object 
 5   peso                 2190 non-null   float64
 6   colesterol           2195 non-null   float64
 7   estado_civil         2185 non-null   object 
 8   id_estado            2195 non-null   int64  
 9   data_coleta          2195 non-null   object 
 10  animal_estimacao     2195 non-null   object 
 11  clima                2195 non-null   object 
 12  bebida_favorita      2195 non-null   object 
 13  hobby                2195 non-null   object 
 14  atividade_fisica     2195 non-null   object 
 15  preferencia_musical  2195 non-null   o